# L13: Lettuce Functions

## Overview
* Lettuce Functions
* Dynamic Scoped Interpreter
* Closures
* Static Scoped Interpreter
* Q&A

## The Lettuce Function

### Function Definitions
* Function Definition Concrete Lettuce Syntax Pattern:
    * function (x) [ e1 ]
* Function Definition Abstract Syntax Pattern:
    * FunDef(Idenitifier,Expr)
* Example:
    * Concrete Syntax:
        * function (a) [ a + 1 ]
    * Abstract Syntax:
        * FunDef("a", Plus(Ident("a"), Const(1.0)))
    * Concrete Value without Closures: 
        * function (a) [ a + 1 ]
    * Abstract Value without Closures:
        * FunDef("a", Plus(Ident("a"), Const(1.0)))

### Function Calls
* Function Call Concrete Lettuce Syntax Pattern:
    * e1(e2)
* Function Call Abstract Syntax Pattern:
    * FunCall(Expr, Expr)
* Example:
    * Concrete Syntax:
        * function (b) [ b + 1 ] ( 2 )
    * Abstract Syntax:
        * FunCall(FunDef("b", Plus(Ident("b"), Const(1.0))), Const(2.0) )
    * Concrete Value without Closures: 
        * 3
    * Abstract Value without Closures:
        * Const(3.0)

### Curry
* Currying:
    * To write a function that takes many inputs, we must curry the function
    * Write a function that takes one input and returns a function as output
* example 1
    * Concrete Syntax:
        * function (c) [ function (d) [ d + c ] ]
    * Abstract Syntax:
        * FunDef("c", FunDef("d", Plus(Ident("d"),Ident("c"))))
    * Concrete Value without Closures: 
        * function (c) [ function (d) [ d + c ] ]
    * Abstract Value without Closures:
        * FunDef("c", FunDef("d", Plus(Ident("d"),Ident("c"))))
* example 2
    * Concrete Syntax:
        * function (c) [ function (d) [ d + c ] ] (1)
    * Abstract Syntax:
        * FunCall(FunDef("c", FunDef("d", Plus(Ident("d"),Ident("c")))), Const(1.0))
    * Concrete Value without Closures: 
        * function (d) [ d + 1 ]
    * Abstract Value without Closures:
        * FunDef("d", Plus(Ident("d"),Const(1.0)))
* example 3
    * Concrete Syntax:
        * function (c) [ function (d) [ d + c ] ] (1)(2)
    * Abstract Syntax:
        * FunCall(FunCall(FunDef("c", FunDef("d", Plus(Ident("d"),Ident("c")))), Const(1.0)), Const(2.0))
    * Concrete Value without Closures: 
        * 3
    * Abstract Value without Closures:
        * Const(3.0)

### Scoping Conditions
* Dynamic vs Static example:
* Concrete Syntax:

~~~
let cat = 2 in [ 
    let foo = function (dog) [ dog + cat ] in [
        let cat = 2000 in [
            foo(3)
]   ]   ]
~~~

* Abstract Syntax:

~~~
Let("cat",
    Const(2.0),
    Let("foo",
        FunDef("dog", Plus(Ident("dog"), Ident("cat"))),
        Let("cat",
            Const(2000.0),
            FunCall(Ident("foo"), Const(3.0))
)   )   )
~~~

#### Dynamic Scope
* Recall, dynamic scope uses the variable environment at each call time to evaluate each call
* used in
    * LISP
    * LaTex
    * BASH
    * It also appears to be in python about Class definitions and Object creation
        * But I don't see why someone would write a python program with such behavior
* concrete syntax:

~~~
let cat = 2 in [ 
    let foo = function (dog) [ dog + cat ] in [
        let cat = 2000 in [
            foo(3)
]   ]   ]
~~~

* Note that at call time of 'foo', the value of 'cat' is 2000
* Concrete Value without Closures: 
    * 2003
* Abstract Value without Closures:
    * Const(2003.0)

#### Static Scope
* recall that static scope uses the variable environment at function defintion to evaluate each call
* Common in most langauge
* concrete syntax:
~~~
let cat = 2 in [ 
    let foo = function (dog) [ dog + cat ] in [
        let cat = 2000 in [
            foo(3)
]   ]   ]
~~~

* Note that at definition time of 'foo', the value of 'cat' is 2
* Concrete Value without Closures: 
    * 5
* Abstract Value without Closures:
    * Const(5.0)

## Interpreter with Dynamic Scope
* You don't need to be able to read this quickly
* I encourage you to look at this again later if you need an example of how dynamic scope could be coded
* To my knowledge, this semester we are not testing on dynamic scoping
* I just want to point out what is causing the issue

### AST

In [1]:
// Same as what you are used to seeing for Lettuce
// but prepended with 'Dyn' for Dynamic scoping...
sealed trait DynExpr
sealed trait DynValue extends DynExpr
type DynIdentifier = String
type DynNumber = Double

case class DynPlus(de1:DynExpr, de2:DynExpr) extends DynExpr
case class DynLet(di1:DynIdentifier, de1:DynExpr, de2:DynExpr) extends DynExpr
case class DynIdent(di1:DynIdentifier) extends DynExpr
case class DynFunCall(e1:DynExpr, e2:DynExpr) extends DynExpr

case class DynFunDef(di1: DynIdentifier, de1:DynExpr) extends DynValue
case class DynConst(dn1:DynNumber) extends DynValue

defined trait DynExpr
defined trait DynValue
defined type DynIdentifier
defined type DynNumber
defined class DynPlus
defined class DynLet
defined class DynIdent
defined class DynFunCall
defined class DynFunDef
defined class DynConst

### Test Setup

In [2]:
// concrete
/*
let cat = 2 in [ 
    let foo = function (dog) [ dog + cat ] in [
        let cat = 2000 in [
            foo(3)
]   ]   ]
*/
val de0:DynExpr = DynLet("cat",
                         DynConst(2.0),
                         DynLet("foo",
                               DynFunDef("dog", DynPlus(DynIdent("dog"),DynIdent("cat"))),
                               DynLet("cat",
                                      DynConst(2000.0),
                                      DynFunCall(DynIdent("foo"), DynConst(3.0))
                                      )
                                )
                         )
// concrete
// 2003
val dv0Expected:DynValue = DynConst(2003.0)            

de0: DynExpr = DynLet(
  "cat",
  DynConst(2.0),
  DynLet(
    "foo",
    DynFunDef("dog", DynPlus(DynIdent("dog"), DynIdent("cat"))),
    DynLet("cat", DynConst(2000.0), DynFunCall(DynIdent("foo"), DynConst(3.0)))
  )
)
dv0Expected: DynValue = DynConst(2003.0)

### Interpreter

In [3]:
def dynEval(de:DynExpr, denv:Map[DynIdentifier, DynValue] = Map()): DynValue = de match {
    case dv:DynValue => dv
    case DynPlus(de1,de2) => dynEval(de1, denv) match {
        case DynConst(dn1) => dynEval(de2, denv) match {
            case DynConst(dn2) => {
                val dnPrime = dn1 + dn2 
                DynConst(dnPrime)
            }   
            case _ => ???  // type error
        }   
        case _ => ???  // type error
    }   
    case DynLet(di1, de1, de2) => {
        val dv1 = dynEval(de1,denv)
        val denvPrime = denv + (di1 -> dv1)
        val dv2 = dynEval(de2, denvPrime)
        dv2 
    }   
    case DynIdent(di1) => denv.getOrElse(di1, throw new IllegalArgumentException("ill formed expression"))
    case DynFunCall(de1, de2) => dynEval(de1, denv) match {
        case DynFunDef(di1, de1) => {
            val dv2 = dynEval(de2, denv)
            val denvPrime = denv + (di1 -> dv2)
            val dv1 = dynEval(de1, denvPrime)
            dv1 
        }   
        case _ => ???  // type error
    }   
}

defined function dynEval

### Test

In [4]:
// val de0:DynExpr = DynLet("cat",
//                          DynConst(2.0),
//                          DynLet("foo",
//                                DynFunDef("dog", DynPlus(DynIdent("dog"),DynIdent("cat"))),
//                                DynLet("cat",
//                                       DynConst(2000.0),
//                                       DynFunCall(DynIdent("foo"), DynConst(3.0))
//                                       )
//                                 )
//                          )
// val dv0Expected:DynValue = DynConst(2003.0)            

val dv0Found:DynValue = dynEval(de0)
if (dv0Found == dv0Expected) {
    println("WOOT WOOT!")
    println("I made an interpreter with dynamic scoping conditions!")
} else {
    println("Uh OH... I failed")
    println(s"EVALUATING: $de0")
    println(s"\tEXPECTED : $dv0Expected")
    println(s"\tFOUND    : $dv0Found")
}

WOOT WOOT!
I made an interpreter with dynamic scoping conditions!


dv0Found: DynValue = DynConst(2003.0)

* What was causing the issue up there in the definiton of dynEval?
* Look at how we complete DynFunCall...
* We use the current environment at the time of function call to evaluate the function body
* THAT causes dynamic scoping conditions!

## Closures
* There are many ways to create static scoping conditions in an interpreter
* Since we are creating a "big-step interpreter" Using closures is a good choice
* A closure is a function with special qualities
* Basically, we don't allow all functions to be values, we only allow closures to be values
* A Closure is a kind of a function definition. It is a function that has a parameter, a function body, and an environment of evaluation
* Concrete Syntax
    * Lettuce will not have a concrete syntax for closures
* Abstract Syntax
    * Closure(Identifier, Expr, Env)

* ex1
    * concrete:
        * function (dog) [ dog + 1 ]
    * abstract expression:
        * FunDef( "dog", Plus(Ident("dog"),Const(1.0)) )
    * abstract value - a closure:
        * Closure( "dog", Plus(Ident("dog"),Const(1.0)), Map() )

* ex2
    * concrete:
        * let cat = 2 in [  function (dog) \[ dog + cat ](3) ]
    * abstract:
        * Let("cat", Const(2.0), FunCall(FunDef("dog", Plus(Ident("dog"),Ident("cat"))), Const(3.0)))
    * concrete value:
        * 5
    * abstract value
        * Const(5.0)
    * Here I will be a bit hand wavey...
    * look at just the function:
        * concrete:
            * function (dog) [ dog + cat ]
        * Abstract:
            * FunDef("dog", Plus(Ident("dog"),Ident("cat")))
        * If we consider the value of cat at the time this function is defined...
        * We see that cat == 2
        * Eventually durring evaluation we create a closure from the function, as follows:
            * Closure("dog", Plus(Ident("dog"),Ident("cat")), Map("cat" -> Const(2.0)))

* ex3
    * concrete:
        * let cat = 2 in [ let foo = function (dog) [ dog + cat ] in [ let cat = 2000 in [ foo(3) ] ] ]
    * abstract:
        * Let("cat", Const(2.0), Let("foo", FunDef("dog", Plus(Ident("dog"),Ident("cat"))), Let("cat", Const(2000.0), FunCall("foo", Const(3.0)))))
    * again... hand wavey...
    * just the function:
        * concrete:
            * function (dog) [ dog + cat ]
        * Abstract:
            * FunDef("dog", Plus(Ident("dog"),Ident("cat")))
        * closure (also abstract):
            * Closure("dog", Plus(Ident("dog"),Ident("cat")), Map("cat" -> 2.0))


* ex3 discussion notes
    * Why is "cat" not mapped to 2000.0?
        * Discuss with peer
        * Tell me why
    * ???

## Static Scoping Interpreter

### Generative Grammar
    * $$\begin{array} &
Program & \rightarrow & TopLevel(Expr) \\
\\
Expr & \rightarrow & Plus(Expr, Expr) \\
& | & Let(Identifier, Expr, Expr) \\
& | & Ident(Identifier) \\
& | & FunDef(Identifier, Expr) \\
& | & FunCall(Expr, Expr) \\
& | & Value \\
\\
Value & \rightarrow & Const(Number) \\
& | & Closure(Identifier, Expr, Env) \\
\\
Identifier & \rightarrow & String \\
Number & \rightarrow & Double \\
Env & \rightarrow & Map[Identifier, Value] \\
\end{array}$$
* But I thought functions are values?
    * Yes, yes, functions are values
        * That a core principle of functional programming
    * But we have a curveball in bigstep interpreters with static scoping
        * We can't actually evaluate all functions on there own
        * and so now only a closure is a value
        * NOTE: closures are a special class of functions
            * they're a function with an evaluation environment attached to them
        * (sad face)

### AST

In [5]:
sealed trait Program
sealed trait Expr
sealed trait Value extends Expr
type Identifier = String
type Number = Double
type Env = Map[Identifier,Value]

case class TopLevel(e0:Expr) extends Program

case class Plus(eLeft:Expr, eRight:Expr) extends Expr
case class Let(x:Identifier, eBind:Expr, eBody:Expr) extends Expr
case class Ident(x:Identifier) extends Expr
case class FunDef(param:Identifier, body:Expr) extends Expr
case class FunCall(hopefullyFunction:Expr, argument:Expr) extends Expr

case class Const(n:Number) extends Value
case class Closure(param:Identifier, body:Expr, environmentAtDeclaration:Env) extends Value

defined trait Program
defined trait Expr
defined trait Value
defined type Identifier
defined type Number
defined type Env
defined class TopLevel
defined class Plus
defined class Let
defined class Ident
defined class FunDef
defined class FunCall
defined class Const
defined class Closure

### Test Setup

In [6]:
// concrete
/*
let cat = 2 in [ 
    let foo = function (dog) [ dog + cat ] in [
        let cat = 2000 in [
            foo(3)
]   ]   ]
*/
val e0:Expr = Let("cat",
                  Const(2.0),
                  Let("foo",
                      FunDef("dog", Plus(Ident("dog"),Ident("cat"))),
                      Let("cat",
                          Const(2000.0),
                          FunCall(Ident("foo"),Const(3.0))
                         )
                     )
                 )
// concrete
// 5
val v0Expected:Value = Const(5.0)            

e0: Expr = Let(
  "cat",
  Const(2.0),
  Let(
    "foo",
    FunDef("dog", Plus(Ident("dog"), Ident("cat"))),
    Let("cat", Const(2000.0), FunCall(Ident("foo"), Const(3.0)))
  )
)
v0Expected: Value = Const(5.0)

### Interpreter

In [7]:
def eval(e:Expr, env:Env = Map()):Value = {
    
    def e2v(e:Expr):Value = eval(e,env)
    
    def e2n(e:Expr):Number = e2v(e) match {
        case Const(n) => n
        case _ => ???  // type error
    }
    
    def applyBinaryArith(e1:Expr, e2:Expr)(f:(Number, Number) => Number):Value = {
        val n1 = e2n(e1)
        val n2 = e2n(e2)
        val nNew = f(n1,n2)
        Const(nNew)
    }
    
    e match {
        case Const(n) => ???
        case Closure(praram, body, environmentAtDeclaration) => ???
        
        case v:Value => ???
        
        case FunDef(param, body) => ???
        
        case FunCall(hopefullyAFunction, argument) => ???
        
        case Plus(e1,e2) => applyBinaryArith(e1,e2){ _ + _ }
        case Let(i1, e1, e2) => {
            val v1 = e2v(e1)
            val envNew = env + (i1 -> v1)
            val v2 = eval(e2, envNew)
            v2
        }
        case Ident(i1) => env.getOrElse(i1, ???)  // ill formed expression
        
        case _ => ???
    }
    
}

defined function eval

### Test

In [8]:
// val e0:Expr = Let("cat",
//                   Const(2.0),
//                   Let("foo",
//                       FunDef("dog", Plus(Ident("dog"),Ident("cat"))),
//                       Let("cat",
//                           Const(2000.0),
//                           FunCall(Ident("foo"),Const(3.0))
//                          )
//                      )
//                  )
// val v0Expected:Value = Const(5.0)   

val v0Found:Value = eval(e0)
if (v0Found == v0Expected) {
    println("WOOT WOOT!")
    println("I made an interpreter with dynamic scoping conditions!")
} else {
    println("Uh OH... I failed")
    println(s"EVALUATING: $e0")
    println(s"\tEXPECTED : $v0Expected")
    println(s"\tFOUND    : $v0Found")
}

: 

## Discussion
* What is a closure?
* Why do closures implement static scoping in the above interpretor?

## Solutions
* I've decided to cut down on my overhead here...
* The solutions are now in the preclass notes
* Please don't read the solutions before or during class... It will spoil my surprises
* After class I will replace the preclass document with the post-class document
* agreed?

In [9]:
// static scoping interpreter
def eval(e:Expr, env:Env = Map()):Value = {
    
    def e2v(e:Expr):Value = eval(e,env)
    
    def e2n(e:Expr):Number = e2v(e) match {
        case Const(n) => n
        case _ => ???  // type error
    }
    
    def applyBinaryArith(e1:Expr, e2:Expr)(f:(Number, Number) => Number):Value = {
        val n1 = e2n(e1)
        val n2 = e2n(e2)
        val nNew = f(n1,n2)
        Const(nNew)
    }
    
    e match {
        case v:Value => v
        case FunDef(i1,e1) => Closure(i1,e1,env)
        
        case FunCall(e1,e2) => e2v(e1) match {
            // NOTE: Here I have "shaddowed" e1 and env
            case Closure(i1,e1,env) => eval(e1,env + (i1 -> e2v(e2)))
            case _ => ???  // type error
        }
        
        case Plus(e1,e2) => applyBinaryArith(e1,e2){ _ + _ }
        case Let(i1, e1, e2) => {
            val v1 = e2v(e1)
            val envNew = env + (i1 -> v1)
            val v2 = eval(e2, envNew)
            v2
        }
        case Ident(i1) => env.getOrElse(i1, ???)  // ill formed expression
        
        case _ => ???
    }
    
}

defined function eval

## Overview
* Lettuce Functions
* Dynamic Scoped Interpreter
* Closures
* Static Scoped Interpreter
* Q&A

## TODOs:
* We have a survey on Moodle that I'd love for you to take:
    * try the link (might work if you're logged into Moodle): https://moodle.cs.colorado.edu/mod/feedback/view.php?id=35105
    * OR go to moodle and search for: "Anonymous Feedback"
* By Rohit's request I created a google survey about recitations. Please take it
    * Sent via email from 'spwi6980@colorado.edu'
* Second Spot Exam is this Friday, 03/01, in Recitation
* The first project is due next Monday, 03/04
* This Thursday we will combine lecture. Meet in the big lecture hall ECCR 265